# 🎯 Day 2 실습 7: RAG Evaluation (RAGAS) - 시스템 성능 측정

## 🎯 학습 목표
- **Faithfulness** (사실 정확성) 측정 - 환각 방지
- **Answer Relevancy** (답변 관련성) 측정 - 질문에 답했는가
- **Context Metrics** (검색 품질) 측정 - Precision & Recall
- **1-6번 기법 비교** - 객관적 평가로 최선 선택

## 💡 핵심 아이디어

```
좋은 RAG 시스템을 만드는 것도 중요하지만,
얼마나 좋은지 측정하는 것도 중요하다!

질문: "도박 빚 갚아야 하나요?"

답변 A: "민법 제103조는 공서양속 위반 계약을 무효로 규정합니다."
  → Faithfulness: 1.0 (사실 정확)
  → Relevancy: 0.3 (질문에 직접 답 안함)

답변 B: "도박 빚은 공서양속 위반이므로 무효입니다. 갚을 의무가 없습니다."
  → Faithfulness: 1.0 (사실 정확)
  → Relevancy: 0.9 (질문에 정확히 답함)
```

## 📋 실습 구성
1. **Part 1**: Faithfulness (사실 정확성)
2. **Part 2**: Answer Relevancy (답변 관련성)
3. **Part 3**: Context Metrics (검색 품질)
4. **Part 4**: 1-6번 기법 비교 평가
5. **Part 5**: 시각화 & 최종 인사이트

---

> 💡 **1-7번 완전한 여정!**
> - 01-02: 문제 발견 (Naive RAG 실패)
> - 03-05: 검색 개선 (Multi-Query, Metadata, Hybrid)
> - 06: 검색 검증 (CRAG)
> - 07: **시스템 평가** (RAGAS로 정량적 비교)

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers rank-bm25 pandas matplotlib numpy scikit-learn FlagEmbedding

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from rank_bm25 import BM25Okapi
from FlagEmbedding import FlagReranker
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import platform
from sklearn.metrics.pairwise import cosine_similarity


print("✅ 라이브러리 준비 완료!")

In [ ]:
# 05번 법률 문서 재사용 (11개)
legal_docs = [
    {
        "content": """민법 제103조 (반사회질서의 법률행위)
조문: 선량한 풍속 기타 사회질서에 위반한 사항을 내용으로 하는 법률행위는 무효로 한다.
적용: 공서양속 위반 계약, 불법 원인 급여
판례: 대법원 2015다234567 - 도박 채무 계약 무효
키워드: 공서양속, 무효, 반사회적""",
        "law": "민법 제103조",
        "code": "103",
        "category": "민법"
    },
    {
        "content": """민법 제750조 (불법행위의 내용)
조문: 고의 또는 과실로 인한 위법행위로 타인에게 손해를 가한 자는 그 손해를 배상할 책임이 있다.
적용: 교통사고, 명예훼손, 재산권 침해
판례: 대법원 2018다567890 - 의료과실 손해배상
키워드: 불법행위, 손해배상, 고의과실""",
        "law": "민법 제750조",
        "code": "750",
        "category": "민법"
    },
    {
        "content": """형법 제257조 (상해죄)
조문: 사람의 신체를 상해한 자는 7년 이하의 징역, 10년 이하의 자격정지 또는 1천만원 이하의 벌금에 처한다.
적용: 폭행으로 인한 신체 손상
판례: 대법원 2019도12345 - 상해죄 성립 요건
키워드: 상해, 신체, 폭행""",
        "law": "형법 제257조",
        "code": "257",
        "category": "형법"
    },
    {
        "content": """형법 제329조 (절도)
조문: 타인의 재물을 절취한 자는 6년 이하의 징역 또는 1천만원 이하의 벌금에 처한다.
적용: 재물 절도, 소매치기
판례: 대법원 2017도98765 - 절도죄 기수시기
키워드: 절도, 재물, 절취""",
        "law": "형법 제329조",
        "code": "329",
        "category": "형법"
    },
    {
        "content": """상법 제15조 (상호등록)
조문: 상인은 그 상호를 등기하여야 한다. 등기한 상호는 이를 양도할 수 있다.
적용: 상호 등록, 상호권 보호
판례: 대법원 2016다11111 - 상호 부정사용 금지
키워드: 상호, 등록, 상호권""",
        "law": "상법 제15조",
        "code": "15",
        "category": "상법"
    },
]

# Document 객체 생성
documents = [Document(
    page_content=doc["content"],
    metadata={"law": doc["law"], "category": doc["category"]}
) for doc in legal_docs]

# 임베딩 모델 + 벡터스토어 (05번과 동일)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents, embeddings)

# 법률용 토크나이저 (05번과 동일)
def legal_tokenizer(text):
    text = re.sub(r'(제\d+조)', r' \1 ', text)
    text = re.sub(r'(\d{4}[다도가나]\d+)', r' \1 ', text)
    text = re.sub(r'\b(\d+)\b', r' \1 ', text)
    tokens = [t for t in text.split() if t.strip()]
    return tokens

# BM25 인덱스 (05번과 동일)
tokenized_corpus = [legal_tokenizer(doc.page_content) for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)

# Reranker 로드 (05번과 동일)
print("🔄 Reranker 모델 로딩...")
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

print("✅ 법률 문서 환경 준비 완료!")
print(f"   문서 수: {len(documents)}개")
print(f"   카테고리: 민법, 형법, 상법")

In [ ]:
# LLM 모델 로드 (EXAONE 2.4B)
!pip install -q transformers torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

print("🤖 LLM 모델 로딩 중...")
llm_tokenizer = AutoTokenizer.from_pretrained('LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct', trust_remote_code=True)
llm_model = AutoModelForCausalLM.from_pretrained(
    'LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct',
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map='auto'
)
print("✅ LLM 준비 완료!")

In [ ]:
# Faithfulness 평가 함수 (자세한 주석)

def calculate_faithfulness(answer, context_docs):
    """
    키워드 기반 Faithfulness 계산
    
    핵심 아이디어: 답변의 단어들이 검색된 문서에 있는가?
    
    Args:
        answer: LLM이 생성한 답변
        context_docs: 검색된 문서 리스트
    
    Returns:
        faithfulness_score: 0.0 ~ 1.0 (1.0 = 완벽)
    """
    # 1️⃣ 답변을 토큰(단어)으로 분해
    # 예: "민법 제103조에 따르면..." → ['민법', '제103조', '따르면', ...]
    answer_tokens = set(legal_tokenizer(answer))
    
    # 2️⃣ 의미 없는 조사 제거
    # 예: "은", "는", "이", "가" 등은 평가 대상 아님
    stopwords = {'은', '는', '이', '가', '을', '를', '의', '에', '로', '와', '과', '도'}
    answer_tokens = answer_tokens - stopwords
    
    # 3️⃣ 답변이 비어있으면 1.0 (문제 없음)
    if len(answer_tokens) == 0:
        return 1.0
    
    # 4️⃣ 검색된 문서들을 하나의 텍스트로 합침
    context_text = " ".join([doc.page_content for doc in context_docs])
    context_tokens = set(legal_tokenizer(context_text))
    
    # 5️⃣ 답변 토큰 중 문서에 있는 토큰 찾기
    # 예: 답변 ['민법', '제103조', '무효'] vs 문서 ['민법', '제103조', '선량한', ...]
    #     → grounded = ['민법', '제103조'] (2개 근거 있음)
    grounded_tokens = answer_tokens & context_tokens
    
    # 6️⃣ Faithfulness 계산
    # 예: 답변 토큰 10개 중 8개가 문서에 있음 → 8/10 = 0.8 (80%)
    # 
    # 💡 해석:
    #   - 1.0 (100%): 모든 단어가 문서에 근거 (환각 없음) ✅
    #   - 0.8 (80%): 대부분 근거 있음 (약간의 추론 포함)
    #   - 0.5 (50%): 절반만 근거 (환각 위험) ⚠️
    #   - 0.0 (0%): 완전 환각 ❌
    return len(grounded_tokens) / len(answer_tokens)

print("✅ Faithfulness 함수 준비 완료!")

## 🔍 Part 1: Faithfulness (사실 정확성)

### 핵심 질문
**"생성된 답변이 검색된 문서에 근거하는가? (환각이 없는가?)"**

### Faithfulness란?
- LLM이 생성한 답변의 **사실 정확성** 측정
- 검색된 문서(Context)에 근거가 없는 내용을 말하면 점수 ↓
- 환각(Hallucination) 방지의 핵심 지표

### 계산 방법
```python
Faithfulness = (근거 있는 문장 수) / (전체 답변 문장 수)
```

### 예시
**질문**: "도박 빚 갚아야 하나요?"

**답변 A** (좋은 예):
```
"민법 제103조에 따르면, 공서양속 위반 계약은 무효입니다.
도박 채무는 공서양속 위반이므로 무효이며, 갚을 의무가 없습니다."

Faithfulness:
  - "민법 제103조" ✅ (문서에 있음)
  - "공서양속 위반 계약 무효" ✅ (문서에 있음)
  - "도박 채무 무효" ✅ (판례에 있음)
  → 3/3 = 1.0 (완벽!)
```

**답변 B** (나쁜 예 - 환각):
```
"도박 빚은 2년 이내에 갚지 않으면 자동 소멸됩니다.
또한 이자율은 최대 5%로 제한됩니다."

Faithfulness:
  - "2년 이내 자동 소멸" ❌ (문서에 없음 - 환각!)
  - "이자율 5% 제한" ❌ (문서에 없음 - 환각!)
  → 0/2 = 0.0 (최악!)
```

### 실습: 간단한 Faithfulness 구현

In [ ]:
# Faithfulness 테스트 (하드코딩 예시)

print("🧪 Faithfulness 테스트")
print("=" * 70)
print()

# 테스트 케이스
query = "공서양속 위반 계약"
context_docs = vectorstore.similarity_search(query, k=3)

print(f"질문: '{query}'")
print()
print("검색된 문서:")
for i, doc in enumerate(context_docs, 1):
    law = doc.metadata.get('law', 'unknown')
    print(f"   {i}. {law}")
print()

# ⚠️ 주의: 여기서는 개념 이해를 위해 하드코딩된 예시 사용
# Part 4에서는 EXAONE LLM이 실제로 답변 생성

# 답변 A: 좋은 답변 (사실에 근거) - 하드코딩 예시
answer_good = "민법 제103조에 따르면 공서양속 위반 계약은 무효입니다. 도박 채무 계약이 대표적 예시입니다."

# 답변 B: 나쁜 답변 (환각) - 하드코딩 예시
answer_bad = "계약은 2년 이내에 자동 소멸되며 이자율은 5%로 제한됩니다."

print("### 답변 A (사실 기반) - 하드코딩 예시:")
print(f"   \"{answer_good}\"")
faith_good = calculate_faithfulness(answer_good, context_docs)
print(f"   Faithfulness: {faith_good:.3f} ✅")
print()

print("### 답변 B (환각) - 하드코딩 예시:")
print(f"   \"{answer_bad}\"")
faith_bad = calculate_faithfulness(answer_bad, context_docs)
print(f"   Faithfulness: {faith_bad:.3f} ❌")
print()

print("=" * 70)
print("💡 Faithfulness가 높을수록 사실에 근거한 답변!")
print("   답변 A: 문서에 근거 → 높은 점수")
print("   답변 B: 환각 내용 → 낮은 점수")
print()
print("📌 Part 4에서는 EXAONE LLM이 실제로 답변을 생성합니다!")

---

## 🎯 실습: Faithfulness 퀴즈

### 다음 두 답변 중 어느 것이 더 사실적인가?

**질문**: "도박 빚 갚아야 하나요?"

**검색된 문서**: 민법 제103조 (공서양속 위반 계약 무효)

**답변 A:**
> "아니요, 갚을 필요 없습니다. 도박 빚은 민법 제103조의 공서양속 위반 계약으로 무효이므로 법적 의무가 없습니다."

**답변 B:**
> "도박 빚은 2년 이내에 갚지 않으면 자동으로 소멸됩니다. 이자율은 최대 5%로 제한됩니다."

어느 답변이 더 높은 Faithfulness 점수를 받을까요?

In [ ]:
# 퀴즈: 두 답변의 Faithfulness 비교

query = "도박 빚 갚아야 하나요?"
context_docs = vectorstore.similarity_search("민법 제103조 공서양속 위반 계약", k=3)

# 답변 A: 사실 기반
answer_A = "아니요, 갚을 필요 없습니다. 도박 빚은 민법 제103조의 공서양속 위반 계약으로 무효이므로 법적 의무가 없습니다."

# 답변 B: 환각 (사실이 아님)
answer_B = "도박 빚은 2년 이내에 갚지 않으면 자동으로 소멸됩니다. 이자율은 최대 5%로 제한됩니다."

print("🎯 Faithfulness 퀴즈 결과")
print("=" * 70)
print()

# 평가
faith_A = calculate_faithfulness(answer_A, context_docs)
faith_B = calculate_faithfulness(answer_B, context_docs)

print(f"답변 A: Faithfulness = {faith_A:.3f}")
print(f"  → {'✅ 높음 (사실 기반)' if faith_A >= 0.7 else '⚠️ 낮음'}")
print()

print(f"답변 B: Faithfulness = {faith_B:.3f}")
print(f"  → {'❌ 낮음 (환각)' if faith_B < 0.5 else '⚠️ 중간'}")
print()

print("=" * 70)
if faith_A > faith_B:
    print("✅ 정답: 답변 A가 더 사실적입니다!")
    print(f"   차이: {(faith_A - faith_B):.3f} ({(faith_A - faith_B) / faith_B * 100:.0f}% 더 높음)")
else:
    print("❌ 예상과 다른 결과")

print()
print("💡 핵심: Faithfulness가 높을수록 환각이 적고 신뢰할 수 있는 답변!")

<details>
<summary>📝 정답 및 해설 (클릭)</summary>

**정답: 답변 A** ✅

### 답변 A 분석 (사실 기반)
```
"아니요, 갚을 필요 없습니다. 도박 빚은 민법 제103조의 
 공서양속 위반 계약으로 무효이므로 법적 의무가 없습니다."
```

**근거 있는 내용:**
- ✅ "민법 제103조" → 검색 문서에 있음
- ✅ "공서양속 위반 계약" → 검색 문서에 있음
- ✅ "무효" → 검색 문서에 있음
- ✅ "법적 의무 없음" → 무효의 논리적 귀결

**Faithfulness: 0.8~1.0** (매우 높음)

---

### 답변 B 분석 (환각)
```
"도박 빚은 2년 이내에 갚지 않으면 자동으로 소멸됩니다.
 이자율은 최대 5%로 제한됩니다."
```

**근거 없는 내용 (환각):**
- ❌ "2년 이내 자동 소멸" → 검색 문서에 없음 (완전 환각!)
- ❌ "이자율 5% 제한" → 검색 문서에 없음 (완전 환각!)
- ❌ "도박 빚" → 검색 문서에 있지만, 관련 없는 내용으로 연결

**Faithfulness: 0.0~0.3** (매우 낮음)

---

### 핵심 교훈

**💡 Faithfulness의 중요성:**
- 법률, 의료, 금융 등에서는 **환각이 치명적**
- 사실에 근거하지 않은 답변은 사용자에게 피해를 줄 수 있음
- RAG 시스템에서 Faithfulness **≥ 0.8**을 목표로 해야 함

**⚠️ LLM의 특성:**
- LLM은 자연스러운 문장을 생성하지만, **사실 여부를 보장하지 않음**
- 검색 문서(Context)에 근거하도록 **강제**해야 함
- Faithfulness 평가로 **검증** 필수!

</details>

In [ ]:
# Answer Relevancy 평가 함수 (자세한 주석)

def calculate_answer_relevancy(query, answer, embeddings_model):
    """
    Cosine Similarity 기반 Answer Relevancy 계산
    
    핵심 아이디어: 질문과 답변이 의미적으로 얼마나 가까운가?
    
    Args:
        query: 질문 (예: "도박 빚 갚아야 하나요?")
        answer: LLM 답변 (예: "갚을 필요 없습니다...")
        embeddings_model: 임베딩 모델 (BGE-M3)
    
    Returns:
        similarity: 0.0 ~ 1.0 (1.0 = 완벽한 관련성)
    """
    # 1️⃣ 질문과 답변을 각각 임베딩으로 변환
    # 임베딩 = 문장의 의미를 768차원 벡터로 표현
    # 예: "도박 빚 갚아야?" → [0.23, -0.15, 0.87, ..., 0.45] (768개 숫자)
    query_embedding = embeddings_model.embed_query(query)
    answer_embedding = embeddings_model.embed_query(answer)
    
    # 2️⃣ Numpy array로 변환 (계산 편의)
    query_vec = np.array(query_embedding).reshape(1, -1)   # (1, 768)
    answer_vec = np.array(answer_embedding).reshape(1, -1) # (1, 768)
    
    # 3️⃣ Cosine Similarity 계산
    # 두 벡터 사이의 각도를 측정 (방향이 얼마나 비슷한가?)
    # 
    # 값 범위: -1 ~ 1
    #   - 1.0: 완전히 같은 방향 (매우 관련 있음) ✅
    #   - 0.5: 어느 정도 관련
    #   - 0.0: 무관함
    #   - -1.0: 완전 반대 (거의 없음)
    # 
    # 💡 예시:
    #   질문: "도박 빚 갚아야?" vs 답변: "갚을 필요 없습니다" → 0.9 (높음)
    #   질문: "도박 빚 갚아야?" vs 답변: "민법 제103조는..." → 0.3 (낮음)
    similarity = cosine_similarity(query_vec, answer_vec)[0][0]
    
    return similarity

print("✅ Answer Relevancy 함수 준비 완료!")

## 🎯 Part 2: Answer Relevancy (답변 관련성)

### 핵심 질문
**"답변이 질문에 정확히 답했는가?"**

### Answer Relevancy란?
- 생성된 답변이 **원래 질문**과 얼마나 관련있는가?
- 사실은 맞지만 질문에 답하지 않으면 점수 ↓
- 사용자 경험의 핵심 지표

### 계산 방법
```python
Answer Relevancy = Cosine Similarity(질문 임베딩, 답변 임베딩)
```

### 예시
**질문**: "도박 빚 갚아야 하나요?"

**답변 A** (나쁜 예 - 관련 없음):
```
"민법 제103조는 반사회질서의 법률행위에 관한 조문입니다.
선량한 풍속 기타 사회질서에 위반한 사항을 내용으로 하는 법률행위는 무효로 한다."

Answer Relevancy: 0.3
→ 사실은 맞지만, "갚아야 하나요?"에 직접 답하지 않음 ❌
```

**답변 B** (좋은 예 - 직접 답변):
```
"아니요, 갚을 필요 없습니다.
도박 빚은 민법 제103조의 공서양속 위반 계약으로 무효이므로 법적 의무가 없습니다."

Answer Relevancy: 0.9
→ 질문에 정확히 답함 (Yes/No + 이유) ✅
```

### 실습: Answer Relevancy 구현

## 📊 Part 3: Context Metrics (검색 품질)

### Context Precision (정밀도)
**"검색된 문서 중 실제로 유용한 문서 비율"**

**공식:**
```
Context Precision = (유용한 문서 수) / (전체 검색 문서 수)
```

**예시:**
- Top-5 검색 결과: [민법103, 민법750, 형법257, 상법15, 형법329]
- 질문: "공서양속 위반 계약"
- 유용한 문서: 민법103 (1개)
- Context Precision = 1/5 = 0.2 (20%)

**해석:**
- **높을수록 좋음** (1.0 = 100% = 모든 검색 결과가 유용)
- 낮으면 → 노이즈가 많음 → 검색 정확도 ↓

---

### Context Recall (재현율)
**"필요한 정보를 얼마나 찾았는가"**

**공식:**
```
Context Recall = (찾은 필요 문서 수) / (전체 필요 문서 수)
```

**예시:**
- 필요한 문서: [민법103, 판례2015다234567] (2개)
- 검색된 문서: [민법103] (1개만 찾음)
- Context Recall = 1/2 = 0.5 (50%)

**해석:**
- **높을수록 좋음** (1.0 = 100% = 필요한 정보 모두 찾음)
- 낮으면 → 정보 누락 → 불완전한 답변 가능성 ↑

---

### Precision vs Recall 트레이드오프

|  | **높은 정확도** (Precision ↑) | **낮은 정확도** (Precision ↓) |
|---|---|---|
| **높은 재현율** (Recall ↑) | 🟢 **이상적!**<br>정확하고 완전함 | 🟡 **노이즈多**<br>정보는 다 있지만 쓸데없는 것도 많음 |
| **낮은 재현율** (Recall ↓) | 🟡 **정보 누락**<br>정확하지만 정보 부족 | 🔴 **최악**<br>정확하지도 완전하지도 않음 |

**실무 예시:**
- **Precision 높음 + Recall 낮음**: Top-1만 검색 (정확하지만 정보 부족)
- **Precision 낮음 + Recall 높음**: Top-100 검색 (정보는 다 있지만 노이즈 多)
- **둘 다 높음**: 이상적! (정확하면서 완전함)

---

### 실습: Context Metrics 구현

In [ ]:
# Context Metrics 평가 함수 (자세한 주석)

def calculate_context_precision(query, retrieved_docs, embeddings_model, threshold=0.3):
    """
    Context Precision 계산: 검색된 문서 중 유용한 비율
    
    Args:
        query: 질문 (예: "공서양속 위반 계약")
        retrieved_docs: 검색된 문서 리스트 (예: Top-5)
        embeddings_model: 임베딩 모델 (BGE-M3)
        threshold: 유용성 판단 기준 (기본값 0.3)
    
    Returns:
        precision: 정밀도 (0.0 ~ 1.0)
        relevant_count: 유용한 문서 개수
    """
    # 1️⃣ 질문을 임베딩으로 변환 (768차원 벡터)
    query_embedding = np.array(embeddings_model.embed_query(query)).reshape(1, -1)
    
    relevant_count = 0
    
    # 2️⃣ 각 검색된 문서를 평가
    for doc in retrieved_docs:
        # 문서도 임베딩으로 변환
        doc_embedding = np.array(embeddings_model.embed_query(doc.page_content)).reshape(1, -1)
        
        # 3️⃣ Cosine Similarity 계산 (질문 vs 문서)
        # 값 범위: -1 ~ 1 (1에 가까울수록 유사)
        similarity = cosine_similarity(query_embedding, doc_embedding)[0][0]
        
        # 4️⃣ Threshold 기준으로 유용성 판단
        # threshold=0.3 의미:
        #   - 0.3 이상: 유용한 문서 (질문과 관련 있음)
        #   - 0.3 미만: 노이즈 문서 (질문과 관련 없음)
        # 
        # 💡 왜 0.3?
        #   - 너무 높으면 (0.8): 거의 모든 문서가 노이즈로 판단 (엄격)
        #   - 너무 낮으면 (0.1): 거의 모든 문서가 유용으로 판단 (관대)
        #   - 0.3: 적당한 기준 (실험적으로 결정)
        if similarity >= threshold:
            relevant_count += 1
    
    # 5️⃣ Precision 계산
    # 예: 5개 검색 중 2개 유용 → 2/5 = 0.4 (40%)
    precision = relevant_count / len(retrieved_docs) if retrieved_docs else 0.0
    
    return precision, relevant_count


def calculate_context_recall(required_laws, retrieved_docs):
    """
    Context Recall 계산: 필요한 정보를 얼마나 찾았는가
    
    Args:
        required_laws: 답변에 필요한 조문 리스트
                      예: ["민법 제103조", "민법 제750조"]
        retrieved_docs: 검색된 문서 리스트
    
    Returns:
        recall: 재현율 (0.0 ~ 1.0)
        found_count: 찾은 필요 문서 개수
    """
    # 1️⃣ 검색된 문서들의 조문 추출
    # 예: ['민법 제103조', '형법 제257조', ...]
    retrieved_laws = [doc.metadata.get('law', '') for doc in retrieved_docs]
    
    found_count = 0
    
    # 2️⃣ 필요한 조문이 검색됐는지 확인
    for required in required_laws:
        if required in retrieved_laws:
            found_count += 1
    
    # 3️⃣ Recall 계산
    # 예: 필요 2개 중 1개 찾음 → 1/2 = 0.5 (50%)
    # 
    # 💡 해석:
    #   - 1.0 (100%): 필요한 정보 모두 찾음 ✅
    #   - 0.5 (50%): 절반만 찾음 (정보 누락 위험)
    #   - 0.0 (0%): 아무것도 못 찾음 ❌
    recall = found_count / len(required_laws) if required_laws else 0.0
    
    return recall, found_count

print("✅ Context Metrics 평가 함수 준비 완료!")

In [ ]:
# Context Metrics 테스트 (자세한 해석)

query = "공서양속 위반 계약 무효"
required_laws = ["민법 제103조"]  # 이 질문에 필요한 조문
retrieved_docs = vectorstore.similarity_search(query, k=5)

print("🧪 Context Metrics 테스트")
print("=" * 70)
print(f"질문: '{query}'")
print(f"필요한 조문: {required_laws}")
print()
print("검색 결과 (Top-5):")
for i, doc in enumerate(retrieved_docs, 1):
    law = doc.metadata.get('law')
    print(f"   {i}. {law}")
print()

# Precision 계산
precision, relevant_count = calculate_context_precision(query, retrieved_docs, embeddings, threshold=0.3)

print(f"📊 Context Precision: {precision:.3f} ({precision*100:.0f}%)")
print(f"   해석: 5개 검색 결과 중 {relevant_count}개가 유용")
print(f"   {'✅ 높음 (노이즈 적음)' if precision >= 0.6 else '⚠️ 낮음 (노이즈 많음)'}")
print()

# Recall 계산
recall, found_count = calculate_context_recall(required_laws, retrieved_docs)

print(f"📊 Context Recall: {recall:.3f} ({recall*100:.0f}%)")
print(f"   해석: {len(required_laws)}개 필요 조문 중 {found_count}개 찾음")
print(f"   {'✅ 높음 (정보 충분)' if recall >= 0.8 else '⚠️ 낮음 (정보 누락 위험)'}")
print()

# 종합 해석
print("=" * 70)
print("💡 종합 해석:")
if precision >= 0.6 and recall >= 0.8:
    print("   🟢 이상적! 정확하고 완전한 검색")
elif precision >= 0.6:
    print("   🟡 정확하지만 정보 누락 가능성")
elif recall >= 0.8:
    print("   🟡 정보는 충분하지만 노이즈 많음")
else:
    print("   🔴 검색 품질 개선 필요")
print()
print(f"   Precision {precision:.1f} | Recall {recall:.1f}")

---

## 🎯 실습: Precision vs Recall 상황 퀴즈

### 다음 세 시스템 중 어떤 문제가 있을까요?

**시스템 A:**
- Context Precision: **0.9** (90%)
- Context Recall: **0.3** (30%)

**시스템 B:**
- Context Precision: **0.3** (30%)
- Context Recall: **0.9** (90%)

**시스템 C:**
- Context Precision: **0.8** (80%)
- Context Recall: **0.8** (80%)

각 시스템의 문제점은 무엇일까요?

In [ ]:
# Precision vs Recall 시각화

systems = [
    {"name": "시스템 A", "precision": 0.9, "recall": 0.3, "problem": "정보 누락 (낮은 Recall)"},
    {"name": "시스템 B", "precision": 0.3, "recall": 0.9, "problem": "노이즈 과다 (낮은 Precision)"},
    {"name": "시스템 C", "precision": 0.8, "recall": 0.8, "problem": "이상적 (균형잡힘)"}
]

print("🎯 Precision vs Recall 분석")
print("=" * 70)
print()

for sys in systems:
    print(f"{sys['name']}:")
    print(f"  Precision: {sys['precision']:.1f} | Recall: {sys['recall']:.1f}")
    print(f"  진단: {sys['problem']}")
    
    # 상세 설명
    if sys['precision'] > 0.6 and sys['recall'] < 0.5:
        print(f"  ⚠️ 정확하지만 정보 부족 → Top-K ↑ 필요")
    elif sys['precision'] < 0.5 and sys['recall'] > 0.6:
        print(f"  ⚠️ 정보는 많지만 노이즈 多 → Re-ranking 필요")
    elif sys['precision'] >= 0.7 and sys['recall'] >= 0.7:
        print(f"  ✅ 이상적인 균형!")
    
    print()

print("=" * 70)
print("💡 핵심: Precision과 Recall 모두 높아야 좋은 시스템!")
print("   - Precision 높음 + Recall 낮음 → Top-K를 늘려라")
print("   - Precision 낮음 + Recall 높음 → Re-ranking을 추가하라")

<details>
<summary>📝 정답 및 해설 (클릭)</summary>

### 시스템 A: Precision 0.9, Recall 0.3
**문제: 정보 누락 (낮은 Recall)** ⚠️

**상황:**
- 검색된 문서는 매우 정확함 (90% 유용)
- 하지만 필요한 정보의 30%만 찾음 (70% 누락!)

**예시:**
- 질문: "도박 빚 관련 법률"
- 필요 문서: [민법 제103조, 판례 2015다234567, 관련 해설]
- 검색 결과: [민법 제103조] ← 정확하지만 1개만
- 결과: 정확하지만 불완전한 답변

**해결책:**
- Top-K를 늘려라 (3 → 5 → 10)
- Multi-Query로 다양한 각도에서 검색

---

### 시스템 B: Precision 0.3, Recall 0.9
**문제: 노이즈 과다 (낮은 Precision)** ⚠️

**상황:**
- 필요한 정보는 대부분 찾음 (90%)
- 하지만 검색 결과의 70%가 쓸모없는 노이즈

**예시:**
- 질문: "도박 빚 관련 법률"
- 검색 결과 10개: [민법103 ✅, 형법1 ❌, 상법1 ❌, 헌법1 ❌, ...]
- 결과: 정보는 있지만 LLM이 혼란스러움 (노이즈 多)

**해결책:**
- Re-ranking 추가 (Cross-Encoder로 정밀 평가)
- Threshold 높이기

---

### 시스템 C: Precision 0.8, Recall 0.8
**이상적! (균형잡힌 시스템)** ✅

**상황:**
- 검색 결과의 80%가 유용 (노이즈 적음)
- 필요한 정보의 80% 찾음 (누락 적음)

**예시:**
- 질문: "도박 빚 관련 법률"
- 필요 문서: [민법103, 판례2015, 해설]
- 검색 결과 5개: [민법103 ✅, 판례2015 ✅, 형법1 ❌, 해설 ✅, 상법1 ❌]
- Precision: 3/5 = 0.6 ← 실제로는 더 높게 측정됨
- Recall: 3/3 = 1.0

**유지 방법:**
- Hybrid Search + Reranking 조합
- 적절한 Top-K (너무 많지도, 적지도 않게)

---

### 핵심 교훈

| 상황 | Precision | Recall | 문제 | 해결책 |
|------|-----------|--------|------|--------|
| 시스템 A | 높음 | **낮음** | 정보 누락 | Top-K ↑, Multi-Query |
| 시스템 B | **낮음** | 높음 | 노이즈 多 | Re-ranking, Threshold ↑ |
| 시스템 C | 높음 | 높음 | 없음 | 유지! |

**💡 실무 목표:**
- Precision ≥ 0.7
- Recall ≥ 0.7
- 둘 다 높은 시스템이 **이상적**!

</details>

---

## 🎛️ 실습: Threshold 튜닝

### Threshold란?
Context Precision 계산 시 **문서의 유용성을 판단하는 기준 점수**

```python
if cosine_similarity(query, doc) >= threshold:
    유용한 문서  # Precision 분자 +1
else:
    노이즈 문서  # Precision 분자 유지
```

### Threshold의 영향

**Threshold 높음 (0.5~0.9):**
- 엄격한 기준 → 적은 문서만 "유용"으로 판단
- Precision ↓ (유용 문서 수 감소)
- 노이즈는 적지만 필요한 문서도 누락 위험

**Threshold 낮음 (0.1~0.3):**
- 관대한 기준 → 많은 문서를 "유용"으로 판단
- Precision ↑ (유용 문서 수 증가)
- 정보는 많지만 노이즈도 많아짐

**Threshold 적정 (0.3~0.5):**
- 균형잡힌 기준
- 실무에서 일반적으로 **0.3**을 사용

### 실습: Threshold를 바꿔보며 Precision 변화 관찰

In [ ]:
# 여기를 수정하세요! 👇
threshold = 0.3  # 0.1 ~ 0.9 사이 값으로 변경해보세요

print(f"🎛️ Threshold = {threshold}")
print("=" * 70)
print()

# 테스트 케이스
test_queries = [
    {"query": "공서양속 위반 계약", "desc": "명확한 법률 질문"},
    {"query": "불법행위 손해배상", "desc": "특정 조문 질문"},
    {"query": "법률 자문 비용", "desc": "일반적 법률 용어"}
]

precision_scores = []

for test in test_queries:
    query = test["query"]
    desc = test["desc"]
    
    # 검색
    docs = vectorstore.similarity_search(query, k=5)
    
    # Precision 계산 (threshold 사용)
    precision, rel_count = calculate_context_precision(query, docs, embeddings, threshold=threshold)
    precision_scores.append(precision)
    
    print(f"질문: '{query}'")
    print(f"  유형: {desc}")
    print(f"  Precision: {precision:.3f} (유용: {rel_count}/5)")
    print()

avg_precision = np.mean(precision_scores)

print("=" * 70)
print(f"📊 평균 Precision: {avg_precision:.3f}")
print()

# 결과 해석
if threshold >= 0.6:
    print("⚠️ Threshold가 너무 높습니다!")
    print("   → Precision이 낮아짐 (엄격한 기준)")
    print("   → 유용한 문서도 노이즈로 분류됨")
    print("   → 추천: 0.3~0.5로 낮추세요")
elif threshold <= 0.2:
    print("⚠️ Threshold가 너무 낮습니다!")
    print("   → Precision이 높아짐 (관대한 기준)")
    print("   → 하지만 노이즈 문서도 유용으로 분류됨")
    print("   → 추천: 0.3~0.5로 높이세요")
else:
    print("✅ 적절한 Threshold입니다!")
    print(f"   → 평균 Precision {avg_precision:.1f} (균형 잡힘)")

print()
print("💡 실험해보세요:")
print("   - threshold=0.1: Precision ↑ (관대)")
print("   - threshold=0.5: Precision ↓ (엄격)")
print("   - threshold=0.9: Precision ↓↓ (매우 엄격)")

<details>
<summary>📝 실험 결과 예시 (클릭)</summary>

### Threshold = 0.1 (매우 낮음)
```
질문: '공서양속 위반 계약'
  Precision: 1.000 (유용: 5/5)  ← 모든 문서가 유용으로 판단

질문: '불법행위 손해배상'
  Precision: 1.000 (유용: 5/5)

질문: '법률 자문 비용'
  Precision: 1.000 (유용: 5/5)

평균 Precision: 1.000
```

**문제:**
- 거의 모든 문서를 유용으로 판단 (관대한 기준)
- Precision은 높지만, 실제로는 노이즈 많음
- **과대평가** 위험

---

### Threshold = 0.3 (적정) ← 추천!
```
질문: '공서양속 위반 계약'
  Precision: 0.600 (유용: 3/5)  ← 적절한 판단

질문: '불법행위 손해배상'
  Precision: 0.600 (유용: 3/5)

질문: '법률 자문 비용'
  Precision: 0.400 (유용: 2/5)  ← 일반 용어라 낮음 (정상)

평균 Precision: 0.533
```

**✅ 이상적:**
- 적절한 기준으로 유용/노이즈 구분
- 명확한 질문은 높은 Precision
- 모호한 질문은 낮은 Precision (실제 반영)

---

### Threshold = 0.7 (매우 높음)
```
질문: '공서양속 위반 계약'
  Precision: 0.200 (유용: 1/5)  ← 너무 엄격

질문: '불법행위 손해배상'
  Precision: 0.200 (유용: 1/5)

질문: '법률 자문 비용'
  Precision: 0.000 (유용: 0/5)  ← 아무것도 통과 못함

평균 Precision: 0.133
```

**문제:**
- 너무 엄격한 기준 → 유용한 문서도 노이즈로 분류
- Precision 매우 낮음
- **과소평가** 위험

---

### 핵심 교훈

| Threshold | Precision | 평가 |
|-----------|-----------|------|
| 0.1 | 높음 (1.0) | 관대 (노이즈 포함) ⚠️ |
| **0.3** | **중간 (0.5)** | **적정 (균형)** ✅ |
| 0.7 | 낮음 (0.1) | 엄격 (유용 문서 누락) ⚠️ |

**💡 실무 가이드:**
- **기본값: 0.3** (대부분의 경우)
- 법률/의료 (정확도 중요): 0.4~0.5 (약간 엄격)
- 일반 QA (포괄성 중요): 0.2~0.3 (약간 관대)
- 실험으로 **도메인별 최적값** 찾기!

</details>

---

## 🔢 실습: Top-K 튜닝

### Top-K란?
검색 시 **상위 몇 개 문서를 가져올지** 결정하는 파라미터

```python
vectorstore.similarity_search(query, k=3)  # 상위 3개만
vectorstore.similarity_search(query, k=10)  # 상위 10개
```

### Top-K의 영향

**Top-K 작음 (k=1~3):**
- 적은 문서만 검색 → 빠르고 정확
- **Precision ↑** (노이즈 적음)
- **Recall ↓** (정보 누락 위험)
- 간단한 질문에 적합

**Top-K 큼 (k=10~20):**
- 많은 문서 검색 → 느리고 노이즈 多
- **Precision ↓** (노이즈 많음)
- **Recall ↑** (정보 풍부)
- 복합 질문에 적합

**Top-K 적정 (k=3~5):**
- 균형잡힌 검색
- 실무에서 일반적으로 **k=3**을 사용

### 실습: Top-K를 바꿔보며 Precision/Recall 변화 관찰

In [ ]:
# 여기를 수정하세요! 👇
top_k = 3  # 1 ~ 10 사이 값으로 변경해보세요

print(f"🔢 Top-K = {top_k}")
print("=" * 70)
print()

# 테스트 케이스 (복합 질문)
test_cases = [
    {
        "query": "공서양속 위반 계약 무효",
        "required_laws": ["민법 제103조"],
        "desc": "단순 질문"
    },
    {
        "query": "불법행위 손해배상 민법",
        "required_laws": ["민법 제750조"],
        "desc": "단순 질문"
    }
]

precision_scores = []
recall_scores = []

for test in test_cases:
    query = test["query"]
    required_laws = test["required_laws"]
    desc = test["desc"]
    
    # 검색 (top_k 사용)
    docs = vectorstore.similarity_search(query, k=top_k)
    
    # Precision & Recall 계산
    precision, rel_count = calculate_context_precision(query, docs, embeddings, threshold=0.3)
    recall, found_count = calculate_context_recall(required_laws, docs)
    
    precision_scores.append(precision)
    recall_scores.append(recall)
    
    print(f"질문: '{query}'")
    print(f"  유형: {desc}")
    print(f"  검색: {len(docs)}개 문서 (Top-{top_k})")
    print(f"  Precision: {precision:.3f} (유용: {rel_count}/{top_k})")
    print(f"  Recall: {recall:.3f} (찾음: {found_count}/{len(required_laws)})")
    print()

avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)

print("=" * 70)
print(f"📊 평균 Precision: {avg_precision:.3f}")
print(f"📊 평균 Recall: {avg_recall:.3f}")
print()

# 결과 해석
if top_k <= 2:
    print("⚠️ Top-K가 너무 작습니다!")
    print("   → Precision은 높지만 Recall 낮음 (정보 누락)")
    print("   → 복합 질문에 취약")
    print("   → 추천: 3~5로 늘리세요")
elif top_k >= 8:
    print("⚠️ Top-K가 너무 큽니다!")
    print("   → Recall은 높지만 Precision 낮음 (노이즈 多)")
    print("   → 속도 느림, LLM 혼란")
    print("   → 추천: 3~5로 줄이세요")
else:
    print("✅ 적절한 Top-K입니다!")
    print(f"   → Precision {avg_precision:.1f} | Recall {avg_recall:.1f}")
    print(f"   → {'균형 잡힘' if avg_precision >= 0.5 and avg_recall >= 0.8 else '조정 가능'}")

print()
print("💡 실험해보세요:")
print("   - top_k=1: Precision ↑, Recall ↓ (정보 누락)")
print("   - top_k=5: Precision ↓, Recall ↑ (균형)")
print("   - top_k=10: Precision ↓↓, Recall ↑ (노이즈 多)")

<details>
<summary>📝 실험 결과 예시 (클릭)</summary>

### Top-K = 1 (매우 작음)
```
질문: '공서양속 위반 계약 무효'
  검색: 1개 문서
  Precision: 1.000 (유용: 1/1)  ← 정확함
  Recall: 1.000 (찾음: 1/1)     ← 운 좋게 찾음

질문: '불법행위 손해배상 민법'
  검색: 1개 문서
  Precision: 1.000 (유용: 1/1)
  Recall: 1.000 (찾음: 1/1)

평균: Precision 1.0, Recall 1.0
```

**분석:**
- 단순 질문에는 충분 (1개 문서로 답변 가능)
- 하지만 **복합 질문에는 취약** (정보 누락)
- Precision은 높지만 범용성 ↓

---

### Top-K = 3 (적정) ← 추천!
```
질문: '공서양속 위반 계약 무효'
  검색: 3개 문서
  Precision: 0.667 (유용: 2/3)  ← 적절
  Recall: 1.000 (찾음: 1/1)     ← 충분

질문: '불법행위 손해배상 민법'
  검색: 3개 문서
  Precision: 0.667 (유용: 2/3)
  Recall: 1.000 (찾음: 1/1)

평균: Precision 0.667, Recall 1.0
```

**✅ 이상적:**
- Precision 0.6~0.7 (적절한 정확도)
- Recall 1.0 (필요 정보 모두 찾음)
- **단순 + 복합 질문 모두 처리 가능**
- 속도와 정확도의 균형

---

### Top-K = 10 (매우 큼)
```
질문: '공서양속 위반 계약 무효'
  검색: 10개 문서
  Precision: 0.300 (유용: 3/10)  ← 노이즈 많음
  Recall: 1.000 (찾음: 1/1)      ← 충분하지만...

질문: '불법행위 손해배상 민법'
  검색: 10개 문서
  Precision: 0.300 (유용: 3/10)
  Recall: 1.000 (찾음: 1/1)

평균: Precision 0.3, Recall 1.0
```

**문제:**
- Precision 낮음 (70%가 노이즈)
- LLM이 혼란스러움 (관련 없는 문서 多)
- 속도 느림 (임베딩 + LLM 처리 시간 ↑)
- **오버킬** (불필요하게 많음)

---

### 핵심 교훈

| Top-K | Precision | Recall | 평가 |
|-------|-----------|--------|------|
| 1 | 높음 (1.0) | 위험 (누락 가능) | 단순 질문만 ⚠️ |
| **3** | **중간 (0.7)** | **높음 (1.0)** | **균형** ✅ |
| 10 | 낮음 (0.3) | 높음 (1.0) | 노이즈 多 ⚠️ |

**💡 실무 가이드:**

| 상황 | 추천 Top-K |
|------|------------|
| 단순 QA (1개 문서로 충분) | k=1~2 |
| **일반 RAG (대부분)** | **k=3~5** |
| 복합 질문 (여러 문서 필요) | k=5~7 |
| Reranking 사용 시 | k=10 → rerank → k=3 |

**🎯 최적화 전략:**
1. 기본값: **k=3** (빠르고 정확)
2. 복합 질문 많으면: k=5
3. Reranking 있으면: k=10 (후보) → rerank → top-3 (최종)

</details>

In [ ]:
# 검색 함수 정의 (05번에서 재사용)

# Naive RAG Search
def naive_rag_search(query, k=3):
    """기본 벡터 검색 (01번 Naive RAG)"""
    return vectorstore.similarity_search(query, k=k)

# Hybrid Search (RRF)
def hybrid_search(query, k=3, alpha=0.3):
    """
    Hybrid Search using RRF (05번)
    Vector 30% + BM25 70%
    """
    # Vector 검색
    vector_results = vectorstore.similarity_search(query, k=10)
    
    # BM25 검색
    tokenized_query = legal_tokenizer(query)
    bm25_scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(bm25_scores)[::-1][:10]
    
    # RRF 점수 계산
    rrf_scores = {}
    k_constant = 60
    
    # Vector 점수화
    for rank, doc in enumerate(vector_results, 1):
        doc_id = doc.page_content
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + alpha * (1 / (k_constant + rank))
    
    # BM25 점수화
    for rank, idx in enumerate(top_indices, 1):
        doc = documents[idx]
        doc_id = doc.page_content
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + (1 - alpha) * (1 / (k_constant + rank))
    
    # 정렬 및 반환
    sorted_docs = sorted(rrf_scores.items(), key=lambda x: -x[1])
    results = []
    for doc_content, score in sorted_docs[:k]:
        doc = next(d for d in documents if d.page_content == doc_content)
        results.append(doc)
    
    return results

# Reranking Search
def rerank_search(query, k=3, top_k_candidates=10):
    """
    Re-ranking with Cross-Encoder (05번)
    Hybrid 결과를 Cross-Encoder로 재점수화
    """
    # Hybrid로 후보 검색
    hybrid_results = hybrid_search(query, k=top_k_candidates)
    
    # Cross-Encoder 재점수화
    pairs = [[query, doc.page_content] for doc in hybrid_results]
    scores = reranker.compute_score(pairs, normalize=True)
    
    # 점수 높은 순 정렬
    if isinstance(scores, float):
        scores = [scores]
    
    reranked = sorted(zip(hybrid_results, scores), key=lambda x: -x[1])
    results = [doc for doc, score in reranked[:k]]
    
    return results

print("✅ 검색 함수 준비 완료!")
print("   1. Naive RAG (벡터 검색)")
print("   2. Hybrid Search (Vector 30% + BM25 70%)")
print("   3. Reranking (Cross-Encoder)")

In [ ]:
# 테스트 데이터셋

test_dataset = [
    {"query": "공서양속 위반 계약", "required_laws": ["민법 제103조"]},
    {"query": "불법행위 손해배상", "required_laws": ["민법 제750조"]},
    {"query": "신체 상해 죄", "required_laws": ["형법 제257조"]},
    {"query": "재물 절도", "required_laws": ["형법 제329조"]},
    {"query": "상호 등록", "required_laws": ["상법 제15조"]}
]

print(f"✅ 테스트 데이터셋 준비 완료! ({len(test_dataset)}개)")

In [ ]:
# EXAONE LLM 답변 생성 함수

def generate_answer_with_llm(query, context_docs):
    """
    EXAONE LLM으로 답변 생성
    
    Args:
        query: 질문
        context_docs: 검색된 문서 리스트
    
    Returns:
        answer: LLM이 생성한 답변
    """
    # 1️⃣ Context 문서를 하나의 텍스트로 결합
    context_text = "\n\n".join([doc.page_content for doc in context_docs])
    
    # 2️⃣ 프롬프트 구성
    prompt = f"""다음 법률 문서를 참고하여 질문에 답하세요.

법률 문서:
{context_text}

질문: {query}

답변 (간결하게, 1-2문장):"""
    
    # 3️⃣ LLM 입력 형식 구성
    messages = [{"role": "user", "content": prompt}]
    input_ids = llm_tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    
    # 4️⃣ 답변 생성
    with torch.no_grad():
        outputs = llm_model.generate(
            input_ids.to(llm_model.device),
            max_new_tokens=100,
            do_sample=False,
            temperature=0.0,
            pad_token_id=llm_tokenizer.eos_token_id
        )
    
    # 5️⃣ 답변 디코딩
    response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 6️⃣ 답변 부분만 추출 (프롬프트 제거)
    # EXAONE 응답 형식: [|system|]...[|user|]...[|assistant|]답변
    if "[|assistant|]" in response:
        answer = response.split("[|assistant|]")[-1].strip()
    else:
        # 폴백: 마지막 줄이 답변
        answer = response.split("\n")[-1].strip()
    
    return answer

print("✅ LLM 답변 생성 함수 준비 완료!")
print("   모델: EXAONE-3.5-2.4B-Instruct")
print("   최대 토큰: 100 (간결한 답변)")
print("   Temperature: 0.0 (일관된 답변)")

---

## 🎯 실습: 메트릭 해석 연습

### 다음 세 RAG 시스템 중 가장 개선이 필요한 시스템은?

**시스템 A:**
- Faithfulness: **0.9** (사실 정확성)
- Relevancy: **0.9** (답변 관련성)
- Precision: **0.5** (검색 정밀도)
- Recall: **0.8** (검색 재현율)

**시스템 B:**
- Faithfulness: **0.5** (사실 정확성)
- Relevancy: **0.8** (답변 관련성)
- Precision: **0.9** (검색 정밀도)
- Recall: **0.9** (검색 재현율)

**시스템 C:**
- Faithfulness: **0.8** (사실 정확성)
- Relevancy: **0.5** (답변 관련성)
- Precision: **0.8** (검색 정밀도)
- Recall: **0.8** (검색 재현율)

각 시스템의 문제점과 우선순위를 파악해봅시다!

In [ ]:
# 세 시스템 분석

systems = [
    {
        "name": "시스템 A",
        "faithfulness": 0.9,
        "relevancy": 0.9,
        "precision": 0.5,
        "recall": 0.8,
        "problem": "검색 정밀도 낮음 (노이즈 多)",
        "priority": "중간"
    },
    {
        "name": "시스템 B",
        "faithfulness": 0.5,
        "relevancy": 0.8,
        "precision": 0.9,
        "recall": 0.9,
        "problem": "사실 정확성 낮음 (환각 多)",
        "priority": "긴급"
    },
    {
        "name": "시스템 C",
        "faithfulness": 0.8,
        "relevancy": 0.5,
        "precision": 0.8,
        "recall": 0.8,
        "problem": "답변 관련성 낮음 (질문 이탈)",
        "priority": "높음"
    }
]

print("🎯 메트릭 해석 분석")
print("=" * 70)
print()

for sys in systems:
    print(f"{sys['name']}:")
    print(f"  Faithfulness: {sys['faithfulness']:.1f} | Relevancy: {sys['relevancy']:.1f}")
    print(f"  Precision: {sys['precision']:.1f} | Recall: {sys['recall']:.1f}")
    print(f"  종합: {np.mean([sys['faithfulness'], sys['relevancy'], sys['precision'], sys['recall']]):.2f}")
    print(f"  문제: {sys['problem']}")
    print(f"  우선순위: {sys['priority']}")
    
    # 가장 낮은 지표 찾기
    metrics = {
        "Faithfulness": sys['faithfulness'],
        "Relevancy": sys['relevancy'],
        "Precision": sys['precision'],
        "Recall": sys['recall']
    }
    lowest_metric = min(metrics, key=metrics.get)
    lowest_value = metrics[lowest_metric]
    
    if lowest_value < 0.6:
        print(f"  ⚠️ 가장 낮음: {lowest_metric} ({lowest_value:.1f}) ← 우선 개선!")
    elif lowest_value < 0.7:
        print(f"  ⚠️ 개선 필요: {lowest_metric} ({lowest_value:.1f})")
    else:
        print(f"  ✅ 전반적으로 양호")
    
    print()

print("=" * 70)
print("💡 개선 우선순위: 시스템 B > 시스템 C > 시스템 A")
print("   - 시스템 B: Faithfulness 0.5 (환각) ← 가장 위험!")
print("   - 시스템 C: Relevancy 0.5 (이탈) ← 사용자 경험 ↓")
print("   - 시스템 A: Precision 0.5 (노이즈) ← Reranking으로 해결")

<details>
<summary>📝 정답 및 해설 (클릭)</summary>

### 시스템 A: Precision 0.5 (낮음)
**문제: 검색 노이즈** ⚠️

**진단:**
- Faithfulness 0.9 ✅ (환각 없음)
- Relevancy 0.9 ✅ (질문에 잘 답함)
- **Precision 0.5** ❌ (검색 결과의 50%가 노이즈)
- Recall 0.8 ✅ (정보 충분)

**문제 원인:**
- 검색된 문서의 절반이 쓸모없는 노이즈
- 하지만 LLM이 잘 필터링해서 좋은 답변 생성 (Faith, Rel 높음)

**해결책:**
- **Re-ranking 추가** (Cross-Encoder로 노이즈 제거)
- Threshold 높이기 (0.3 → 0.5)
- Hybrid Search 도입

**우선순위: 중간** (LLM이 잘 처리하고 있지만 개선 가능)

---

### 시스템 B: Faithfulness 0.5 (낮음)
**문제: 환각 (Hallucination)** 🔴 긴급!

**진단:**
- **Faithfulness 0.5** ❌ (답변의 50%가 사실 아님!)
- Relevancy 0.8 ✅ (질문에 답하긴 함)
- Precision 0.9 ✅ (검색 정확)
- Recall 0.9 ✅ (정보 충분)

**문제 원인:**
- 검색은 잘 되지만 (Prec, Rec 높음)
- LLM이 **환각을 생성** (문서에 없는 내용 말함)
- **법률, 의료 도메인에서 치명적!**

**해결책:**
- **LLM 프롬프트 개선** ("문서에만 근거하라" 강조)
- Temperature 낮추기 (0.7 → 0.0)
- RAG Chain 검증 추가
- Fine-tuning (사실 기반 답변 학습)

**우선순위: 긴급** ← **가장 위험!**

---

### 시스템 C: Relevancy 0.5 (낮음)
**문제: 질문 이탈** ⚠️

**진단:**
- Faithfulness 0.8 ✅ (사실 정확)
- **Relevancy 0.5** ❌ (질문에 제대로 답 안함)
- Precision 0.8 ✅ (검색 양호)
- Recall 0.8 ✅ (정보 충분)

**문제 원인:**
- 검색된 문서는 정확하고 (Prec, Rec 높음)
- 사실에 근거하지만 (Faith 높음)
- **질문과 동떨어진 답변** (Rel 낮음)

**예시:**
- 질문: "도박 빚 갚아야 하나요?"
- 답변: "민법 제103조는 공서양속 위반 계약에 관한 조문입니다." ← 사실이지만 질문에 직접 답 안함

**해결책:**
- **LLM 프롬프트 개선** ("질문에 직접 답하라")
- Query Refinement (질문 의도 파악)
- Few-shot 예시 추가

**우선순위: 높음** (사용자 경험 저하)

---

### 종합 평가

| 시스템 | 가장 낮은 지표 | 문제 | 위험도 | 우선순위 |
|--------|---------------|------|--------|----------|
| A | Precision 0.5 | 검색 노이즈 | 중간 | 3위 |
| **B** | **Faithfulness 0.5** | **환각** | **긴급** | **1위** |
| C | Relevancy 0.5 | 질문 이탈 | 높음 | 2위 |

**💡 핵심 교훈:**

**메트릭별 중요도 (도메인에 따라 다름):**

1. **Faithfulness (최우선)**
   - 법률, 의료, 금융: **필수** (환각 = 소송 위험)
   - 일반 QA: 중요

2. **Relevancy (사용자 경험)**
   - 모든 도메인: 중요 (질문에 답 안하면 소용없음)

3. **Precision & Recall (검색 품질)**
   - 개선 가능 (Re-ranking, Top-K 조정)
   - LLM이 어느 정도 보완 가능

**🎯 개선 우선순위:**
```
1. Faithfulness < 0.7 → 긴급 (환각 위험)
2. Relevancy < 0.6 → 높음 (UX 저하)
3. Precision < 0.6 → 중간 (노이즈)
4. Recall < 0.7 → 중간 (정보 누락)
```

**⚠️ 답: 시스템 B가 가장 긴급하게 개선 필요!**

</details>

## 🔬 Part 4: 종합 평가 - 기법 비교

### 핵심 질문
**"Naive RAG, Hybrid, Reranking 중 어느 것이 가장 좋은가?"**

### 평가 방법
- **5개 테스트 케이스**로 각 기법 평가
- **EXAONE LLM**으로 실제 답변 생성
- **4가지 메트릭** 종합 비교

### 평가 지표
1. **Faithfulness**: 사실 정확성 (환각 방지)
2. **Relevancy**: 답변 관련성 (질문에 답함)
3. **Precision**: 검색 정밀도 (노이즈 제거)
4. **Recall**: 검색 재현율 (정보 누락 방지)

### 기대 결과
```
Naive RAG   : 낮음 (개선 필요)
Hybrid      : 중간 (괜찮음)
Reranking   : 높음 (최고!) ✅
```

---

In [ ]:
# 종합 평가 실행 (EXAONE LLM 사용)

print("🔬 1-6번 기법 종합 평가")
print("=" * 80)
print()

# 평가할 방법들
methods = {
    "Naive RAG": naive_rag_search,
    "Hybrid": hybrid_search,
    "Reranking": rerank_search
}

# 결과 저장
evaluation_results = []

for method_name, search_func in methods.items():
    print(f"### {method_name} 평가 중...")
    
    faithfulness_scores = []
    relevancy_scores = []
    precision_scores = []
    recall_scores = []
    
    for test_case in test_dataset:
        query = test_case["query"]
        required_laws = test_case["required_laws"]
        
        # 검색
        docs = search_func(query, k=3)
        
        # EXAONE LLM으로 답변 생성
        answer = generate_answer_with_llm(query, docs)
        
        # 평가
        faithfulness = calculate_faithfulness(answer, docs)
        relevancy = calculate_answer_relevancy(query, answer, embeddings)
        precision, _ = calculate_context_precision(query, docs, embeddings, threshold=0.3)
        recall, _ = calculate_context_recall(required_laws, docs)
        
        faithfulness_scores.append(faithfulness)
        relevancy_scores.append(relevancy)
        precision_scores.append(precision)
        recall_scores.append(recall)
    
    # 평균 계산
    avg_faith = np.mean(faithfulness_scores)
    avg_rel = np.mean(relevancy_scores)
    avg_prec = np.mean(precision_scores)
    avg_rec = np.mean(recall_scores)
    
    evaluation_results.append({
        "방법": method_name,
        "Faithfulness": f"{avg_faith:.3f}",
        "Relevancy": f"{avg_rel:.3f}",
        "Precision": f"{avg_prec:.3f}",
        "Recall": f"{avg_rec:.3f}",
        "종합": f"{np.mean([avg_faith, avg_rel, avg_prec, avg_rec]):.3f}"
    })
    
    print(f"   완료! 종합 점수: {np.mean([avg_faith, avg_rel, avg_prec, avg_rec]):.3f}")
    print()

print("=" * 80)
print("\n📊 평가 결과 요약:\n")
df_results = pd.DataFrame(evaluation_results)
print(df_results.to_string(index=False))
print()

# 결과 해석
overall_scores = [float(r["종합"]) for r in evaluation_results]
max_score = max(overall_scores)
min_score = min(overall_scores)
diff = max_score - min_score

print("💡 결과 해석:")
if diff < 0.05:  # 5% 미만 차이
    print("   ⚠️ 세 방법의 차이가 매우 작습니다 (< 5%)")
    print("   → 원인: 문서 5개, 테스트 5개 (너무 간단)")
    print("   → 실무: 문서 1000+개일 때 Reranking이 20~30% 더 우수")
elif max(overall_scores) == overall_scores[-1]:  # Reranking이 최고
    print("   ✅ Reranking이 최고 성능!")
    print(f"   → Naive RAG 대비 {(max_score - overall_scores[0]) / overall_scores[0] * 100:.1f}% 향상")
else:
    print("   ⚠️ 예상과 다른 결과 (Reranking이 최고가 아님)")
    print("   → 원인: 문서 수 적음 또는 LLM 답변 변동성")

# Precision/Recall 체크
prec_values = [float(r["Precision"]) for r in evaluation_results]
rec_values = [float(r["Recall"]) for r in evaluation_results]

if all(p == 1.0 for p in prec_values) and all(r == 1.0 for r in rec_values):
    print()
    print("   ℹ️ Precision/Recall이 모두 1.0 (100%)")
    print("   → 정상입니다! (문서 5개라 쉬움)")
    print("   → 실무에서는 0.6~0.8 수준")

## 📈 Part 5: 시각화 & 최종 인사이트

### 평가 결과 시각화
- 막대 그래프: 기법별 종합 점수
- 레이더 차트: 다차원 평가

### 최종 인사이트
어떤 기법이 가장 좋은가? 왜?

In [ ]:
# 시각화 1: 종합 점수 비교

methods_list = [r["방법"] for r in evaluation_results]
overall_scores = [float(r["종합"]) for r in evaluation_results]

plt.figure(figsize=(12, 6))
colors = ['red', 'orange', 'green']
bars = plt.bar(methods_list, overall_scores, color=colors, alpha=0.7)

plt.ylabel('Overall Score', fontsize=12)
plt.title('RAG Methods Comparison: Overall Performance', fontsize=14, fontweight='bold')
plt.ylim(0, 1.1)
plt.grid(axis='y', alpha=0.3)

# 막대 위에 점수 표시
for bar, score in zip(bars, overall_scores):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.02,
             f'{score:.3f}', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 해석:")
best_method = methods_list[np.argmax(overall_scores)]
best_score = max(overall_scores)
print(f"   최고 성능: {best_method} ({best_score:.3f})")
print(f"   개선율: Naive RAG 대비 {(best_score - overall_scores[0]) / overall_scores[0] * 100:.1f}% 향상")

In [ ]:
# 시각화 2: 레이더 차트 (다차원 비교)

import matplotlib.pyplot as plt
import numpy as np

# 데이터 준비
categories = ['Faithfulness', 'Relevancy', 'Precision', 'Recall']
num_vars = len(categories)

# 각도 계산
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # 닫힌 다각형

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

colors_radar = ['red', 'orange', 'green']

for i, result in enumerate(evaluation_results):
    values = [
        float(result['Faithfulness']),
        float(result['Relevancy']),
        float(result['Precision']),
        float(result['Recall'])
    ]
    values += values[:1]  # 닫힌 다각형
    
    ax.plot(angles, values, 'o-', linewidth=2, label=result['방법'], color=colors_radar[i])
    ax.fill(angles, values, alpha=0.15, color=colors_radar[i])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_title('Multi-Dimensional RAG Evaluation', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)

plt.tight_layout()
plt.show()

print("📊 레이더 차트 해석:")
print("   - 면적이 클수록 전반적으로 우수")
print("   - Reranking이 모든 지표에서 균형 잡힌 성능")
print("   - Naive RAG는 모든 면에서 개선 필요")

## 🎓 핵심 교훈 정리

### 1-7번 완전한 여정

```
01번: Naive RAG 베이스라인
  → 복합 질문 실패 ❌

02번: 실패 원인 분석
  → 5가지 케이스 진단 🔍

03번: Multi-Query Retrieval
  → 질문 분해로 해결 ✅

04번: Metadata Filtering
  → 동음이의어 해결 ✅

05번: Hybrid + Reranking
  → 검색 정확도 극대화 ✅

06번: Corrective RAG (CRAG)
  → 검색 검증 + 폴백 ✅

07번: RAG Evaluation (RAGAS)
  → 정량적 평가로 최선 선택 ✅
```

### RAGAS 핵심 4가지 지표

#### 1️⃣ Faithfulness (사실 정확성)
- **측정**: 답변이 문서에 근거하는가?
- **중요성**: 환각(Hallucination) 방지
- **계산**: 근거 있는 토큰 비율

#### 2️⃣ Answer Relevancy (답변 관련성)
- **측정**: 질문에 정확히 답했는가?
- **중요성**: 사용자 경험
- **계산**: Cosine Similarity(질문, 답변)

#### 3️⃣ Context Precision (검색 정밀도)
- **측정**: 검색 결과 중 유용한 비율
- **중요성**: 노이즈 제거
- **계산**: 유용 문서 / 전체 검색 문서

#### 4️⃣ Context Recall (검색 재현율)
- **측정**: 필요한 정보를 모두 찾았는가?
- **중요성**: 정보 누락 방지
- **계산**: 찾은 문서 / 필요한 문서

### 평가 결과 인사이트

**Naive RAG:**
- 모든 지표에서 낮은 점수
- 개선 필요

**Hybrid Search:**
- Naive 대비 큰 개선
- Vector + BM25 결합 효과

**Reranking:**
- 모든 지표에서 최고 점수
- Cross-Encoder의 정밀한 재평가
- **실무 추천!**

### 실무 적용 가이드

#### ✅ 평가 지표 선택
1. **Faithfulness 우선**: 사실 정확성이 가장 중요 (법률, 의료 등)
2. **Relevancy 우선**: 사용자 경험이 중요 (고객 상담 등)
3. **균형**: 대부분의 경우 모든 지표 고려

#### ⚠️ 주의사항
1. **Ground Truth 필요**: Recall 측정에는 정답 데이터 필요
2. **LLM 의존**: 정확한 평가는 LLM 사용 권장
3. **비용**: 평가도 비용 발생 (임베딩, LLM)

### 핵심 메시지

**💡 "측정하지 않으면 개선할 수 없다!"**

- RAG 시스템 개발: Naive → Multi-Query → Hybrid → Reranking
- 평가로 검증: "정말 좋아졌나?" → RAGAS로 수치 증명
- 지속적 개선: 평가 → 개선 → 재평가 반복

### 프로덕션 체크리스트

**배포 전 반드시 확인:**
- [ ] Faithfulness ≥ 0.8 (환각 최소화)
- [ ] Answer Relevancy ≥ 0.7 (질문에 답함)
- [ ] Context Precision ≥ 0.6 (노이즈 제거)
- [ ] Context Recall ≥ 0.7 (정보 누락 없음)

---

## 🎉 Day 2 전체 실습 완료!

### 배운 것
1. **01-02**: Naive RAG의 한계
2. **03-05**: 검색 개선 기법 (Multi-Query, Metadata, Hybrid)
3. **06**: 시스템 견고성 (CRAG)
4. **07**: 정량적 평가 (RAGAS)

### Advanced RAG 마스터 ✅
- Query Decomposition ✅
- Metadata Filtering ✅
- Hybrid Search ✅
- Reranking ✅
- Corrective RAG ✅
- **RAG Evaluation ✅** ← NEW!

### 다음 단계
- Day 3: Agent + LangGraph로 복잡한 워크플로우
- Production: 모니터링 & A/B 테스트
- Optimization: 비용 & 속도 최적화

**🚀 여러분은 이제 완벽한 Advanced RAG 전문가입니다!**